In [1]:
# Thank you for viewing my project! All functions and code that are not marked as being made by a group member are my own work.

# imports corpus and random and counter tools
import nltk                                                                       
from collections import Counter
import random

nltk.download('brown')
from nltk.corpus import brown
# gets POS tags
nltk.download('universal_tagset')

# categories of brown corpus user can choose from
genres = ["news", "editorial", "reviews", "religion", "hobbies", "lore", "belles_lettres", "government", "learned", "fiction", "mystery", "science_fiction", "adventure", "romance", "humor"]
# starts dialogue w/ user where they pick genre of poem
print("Hi there! Here are your options for the genre of poem: ")
print("news\neditorial\nreviews\nreligion\nhobbies\nlore\nbelles_lettres\ngovernment\nlearned\nfiction\nmystery\nscience_fiction\nadventure\nromance\nhumor")
genre = ""
genre = input("Which genre would you like? (Make sure to spell it right!) ")
genre = genre.lower()
# handles misspelled/invalid input
while genre not in genres or genre == "":
  genre = input("Sorry that's not an option. Check your spelling and try again: ")
print("Great choice! It'll just be a minute.")
# creates tagged corpus
corpus = brown.tagged_words(categories = genre, tagset = 'universal') 

# this function was created by a group member
def find_rhyme_pos(string1):                                                      
    """
    Uses vowels to find the position of the rhyme in a word.
    
    Argument
    ---------
    string1: line of poetry
    
    Returns
    ---------
    length of the last word starting from the last vowel.
    
    Example
    ---------
    find_rhyme_pos("chicken")
    >>>2
    """

    vowels = ['a', 'e', 'i', 'o', 'u']
    for n in range(1, len(string1) - 1):
        if string1[-n] in vowels and string1[-(n+1)] not in vowels:
            return n
    return len(string1)

# this function was created by a group member
def rhymes(string1, string2): # checks rhyme                                      
    """
    Checks if two words rhyme.
    
    Arguments
    ---------
    string1: first word
    string2: second word
    
    Returns
    ---------
    Bool
    True if they rhyme, False otherwise.
    
    Example
    ---------
    rhymes("block", "shock")
    >>>True
    """
    # compares the section of the word where the rhyme is to that of the other word 
    return string1[-find_rhyme_pos(string1):] == string2[-find_rhyme_pos(string2):]

# this function was created by a group member
def construct_rhyme_dict(corpus): # makes whole dictionary of rhymes              
    """
    Creates a dictionary of rhymes.
    
    Argument
    ---------
    corpus: the subset of the brown corpus chosen by the user
    
    Returns
    ---------
    A dictionary with a word as a key and words that rhyme with that word as its values.
    
    Example
    ---------
    construct_rhyme_dict("Friday")
    >>>[('may', 'VERB'), ('Wednesday', 'NOUN'), ('Tuesday', 'NOUN'), ('Highway', 'NOUN'), ('Tuesday', 'NOUN'), ('Monday', 'NOUN'), ('highway', 'NOUN'), ('Monday', 'NOUN'), ('highway', 'NOUN'), ('Highway', 'NOUN'), ('highway', 'NOUN'), ('Highway', 'NOUN')]
    """
    rhyme_dict = {}
    for item1 in corpus:
        rhyme_dict[item1[0]] = []
        for item2 in corpus:
            if rhymes(item1[0], item2[0]) and item1 != item2:
                rhyme_dict[item1[0]].append(item2)
    return rhyme_dict

# lists for pos_dict
nouns = []
verbs = []
dets = []
adj = []
bigrams = []

# dictionary that simplifies code
pos_dict = {'NOUN': nouns, 'ADJ': adj, 'VERB': verbs, 'DET': dets}

def pick_word(pos, syllables = 0):
    
    """
    Picks random word based on POS and syllables.
    
    Arguments
    ---------
    pos: part of speech needed in template.
    syllables: amount of syllables allowed in a particular area in the template.
    
    Returns
    ---------
    string
    
    Example
    ---------
    pick_word("ADJ")
    >>>"pretty"
    """

    for i in range(len(pos_dict[pos])):
      # picks random value from pos_dict based on POS key
        word = random.choice(pos_dict[pos])
        # checks syllable count
        if syllables != 0 and syllable_count(word) == syllables:
            return word
    return random.choice(pos_dict[pos])

def bigram_probabilities(word, pos, syllables): # returns most probable word to come before, checks that its POS is correct
    """
    Returns most probable word to come before a word that fits POS and syllable count given in template
    
    Arguments
    ---------
    word: current word in the line.
    pos: part of speech that the word before must be.
    syllables: syllables allowed for that word.
    
    Returns
    ---------
    string
    
    Example
    ---------
    bigram_probabilities("block", 'NOUN', 2)
    >>>"human"
    """
    completions = []
    for pair in bigrams:
      # if second word of bigram equals current word
      if pair[1][0] == word: 
          # if pos of first word in bigram equals part of speech needed
          if pair[0][1] == pos:
            # first word in bigram is number of syllables needed
            if syllable_count(pair[0][0]) == syllables: 
              completions.append(pair[0][0])
    # if no words fit completion, return one w/ correct pos and syllable count:
    if completions==[]: 
      return pick_word(pos, syllables)
    completions = Counter(completions)
    return (completions.most_common(1))[0][0]

def gather_words(): 
    """
    Adds words to pos_dict based on their POS tag
    
    Example
    ---------
    gather_words()
    >>>pos_dict['NOUNS'] = ["dog", "cat", ...]
      
    """
    for word in range(len(corpus)-1): # loops through corpus
        # if POS marker of word is a category in pos_dict
        if pos_dict.get(corpus[word][1]) != None:
          # add that word to the respective category in pos_dict
          pos_dict[corpus[word][1]].append(corpus[word][0]) 
    # make all but nouns lowercase
    for word in pos_dict['VERB']:
      word.lower()
    for word in pos_dict['DET']:
      word.lower()
    for word in pos_dict['ADJ']:
      word.lower()

def bigram_collector(corpus):
    """
    Creates list of word bigrams.
    
    Arguments
    ---------
    corpus: list
        the subset of the brown corpus chosen by the user.
    
    Returns
    ---------
    bigrams: list
    
    Example
    ---------
    bigram_collector(corpus)
    >>>[(('The', 'DET'), ('Fulton', 'NOUN')),
        (('Fulton', 'NOUN'), ('County', 'NOUN')),
        (('County', 'NOUN'), ('Grand', 'ADJ')),
        (('Grand', 'ADJ'), ('Jury', 'NOUN')),
    """
    for n in range(len(corpus)-1):
      # makes pairs of adjacent words
        pair = (corpus[n], corpus[n+1])
        bigrams.append(pair)
    return bigrams

# this function was created by a group member
def syllable_count(word):                                                         
    """
    Counts number of syllables in word.
    
    Arguments
    ---------
    word: str
        a word in a line
        
    Returns
    ---------
    count: int
        number of syllables
    
    Example:
    ---------
    syllable_count("chicken")
    >>> 2
    
    """
    vowels = ['a', 'e', 'i', 'o', 'u', 'y', 'A', 'E', 'I', 'O', 'U', 'Y']
    count = 0
    boolean = True
    for char in range(0, len(word)):
      # if letter isn't vowel nothing changes
        if word[char] not in vowels:
            boolean = True
        # if vowel 'e' is at end of word
        elif boolean and word[char] == 'e' and char == len(word) - 1:
            # if word longer than 2 letters and 3rd-to-last letter not a vowel, or if word is 2 letters or less, add a syllable
            # accounts for silent 'e' at end of word, or clusters before a word-terminal 'e' in words like 'debacle' 
            if (len(word) > 2 and word[char - 2] not in vowels) or len(word) <= 2:
                count += 1
        # elif still True and letter is vowel, add syllable
        # boolean becomes False so two adjacent vowels aren't counted as separate syllables
        elif boolean and word[char] in vowels:
            count += 1
            boolean = False
    return count

gather_words() # makes categories of words, listed by pos
bigram_collector(corpus) # makes list of bigrams
rhyme_dict = construct_rhyme_dict(corpus[:2000]) # make rhyme corpus
print("Thanks for waiting! Now run the cell below to generate some poems!")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
Hi there! Here are your options for the genre of poem: 
news
editorial
reviews
religion
hobbies
lore
belles_lettres
government
learned
fiction
mystery
science_fiction
adventure
romance
humor
Which genre would you like? (Make sure to spell it right!) belles_lettres
Great choice! It'll just be a minute.
Thanks for waiting! Now run the cell below to generate some poems!


In [2]:
def write_poem(template, num_lines):
    """
    Writes the entire poem.
    
    Arguments
    ---------
    template: pre-defined templates that contain part of speech and syllable allowance.
    num_lines: how many lines each poem will be, obtained from user input
    
    Returns
    ---------
    lists within a list
    
    Example
    ---------
    write_poem(template, num_lines)
    >>>
    The City 

    any John was ghastly dress 

    real need felt better infection 

    a good sound coming Anthea 

    any speech made fragile parents 

    the new Rose gambling Devil 
    """
    poem = []
    write_title()
    print('\n')
  
    for n in range(num_lines):
      line = []
      if poem == []: # if we're on the first line
        line = write_line(template[random.randint(0,3)], pick_word('NOUN')) # write first line
      else: # if we're on the second or more line
        if previous_line[-1:][0] in rhyme_dict: # if word at end of line is in the rhyme_dict
          rhyming_words = rhyme_dict[previous_line[-1:][0]] # lists words that rhyme
          if rhyming_words != []: # if there are words that rhyme w/ the last word of previous line
            for word in rhyming_words:
              if word[1] == 'NOUN': # if one of those words is a noun, use it to write next line
                line = write_line(template[random.randint(0,3)], word[0])
              else: # pick random noun to write next line with
                line = write_line(template[random.randint(0,3)], pick_word('NOUN')) 
          else: # if no rhyming words, pick random noun to write next line with
            line = write_line(template[random.randint(0,3)], pick_word('NOUN')) 
        else:
          line = write_line(template[random.randint(0,3)], pick_word('NOUN'))
      poem.append(line) # add to poem
      previous_line = line # makes first line into previous to check rhyme
    
    # fixes formatting of poem
    for line in poem:
        for word in line:
            print(word, end = " ")
        print('\n')

def write_line(sentence, last_word): # takes one of the line templates and the last word of the line
    """
    Writes the lines of the poem.
    
    Arguments
    ---------
    sentence: str
    last_word: str
        the last word of the line
    
    Returns
    ---------
    line: list
    
    Example
    ---------
    write_line(sentence, last_word)
    >>>any speech made fragile parents
    """
    
    line=[]
    fourth_word = (bigram_probabilities(last_word, sentence[-2:][0][0], sentence[-2:][1][1])) # picks next four words
    third_word = (bigram_probabilities(fourth_word, sentence[-3:][0][0], sentence[-3:][1][1]))
    second_word = (bigram_probabilities(third_word, sentence[-4:][0][0], sentence[-4:][1][1]))
    first_word = (bigram_probabilities(second_word, sentence[-5:][0][0], sentence[-5:][1][1]))

    line.append(first_word) # adds four words to line in order
    line.append(second_word)
    line.append(third_word)
    line.append(fourth_word)
    line.append(last_word)
    return line

def write_title():
    """
    Writes a title for the poem.
    
    Example
    ---------
    write_title()
    >>>The City
    """
    title_templates = [[('DET'), ('NOUN')], [('DET'), ('ADJ'), ('NOUN')], [('ADJ'), ('NOUN')], [('NOUN')], [('ADJ')], [('VERB')]]
    title = []
    for word in title_templates[random.randint(0,5)]: # picks words for each pos in title
        title.append(pick_word(word).capitalize())

    for word in title: # prints nicely
        print(word, end = " ")

# different patterns for lines of poem
template = [[('DET', 1), ('NOUN', 2), ('VERB', 1), ('ADJ', 1), ('NOUN', 2)],
                [('NOUN', 1), ('VERB', 1), ('DET', 1), ('ADJ', 2), ('NOUN', 2)],
                [('DET', 1), ('ADJ', 1), ('NOUN', 1), ('VERB', 1), ('NOUN', 2)], 
                [('ADJ', 2), ('NOUN', 1), ('VERB', 1), ('ADJ', 1), ('NOUN', 2)]]

num_poems = input("How many poems would you like? Enter an integer: ")
# handle decimals and make an integer
num_poems = float(num_poems)
num_poems = int(num_poems)

if num_poems == 1:
  num_lines = input("How many lines would you like your poem to be? Enter an integer: ")
  print("Okay, here's your poem:")
elif num_poems == 0:
  print("Okay, never mind.")
else:
  num_lines = input("How many lines should each poem have? Enter an integer: ")
  print("Okay, here are your poems: ")

# handle decimals and make an integer
num_lines = float(num_lines)
num_lines = int(num_lines)

for i in range(num_poems):
    print(str(i + 1) + ".\n")
    write_poem(template, num_lines)

How many poems would you like? Enter an integer: 4
How many lines should each poem have? Enter an integer: 11
Okay, here are your poems: 
1.

What Society 

guests make any large domes 

his own home makes use 

friends bought nether striking example 

any one is simple goodness 

the Old One serves arms 

a slow North embrace butlers 

the free world responds traces 

least one is northern genius 

a strong chaos remain Hindu 

guests make any banal import 

cold war be major effect 

2.

Boulevard 

any work was happy Crowder 

cold war be chalk-white rooms 

the same time joining Europe 

any way been present fears 

least one is precious North 

any French were other end 

guests make any entire organization 

a young girl learned prison 

any Thorp am human condition 

least one is little orphans 

any South build social status 

3.

Furious 

the real life follow Aristotle 

a new form decide Coolidge 

the great Lear believe hypothesis 

the same way employ hulk 

any men are sp